In [1]:
# Downloading required libraries
!pip install -q transformers datasets sentencepiece tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.1 MB/s eta 0:00:00


In [2]:
# Deleting the cache content of the transformer namespacename paced 
!rm -rf HOME/.cache/huggingface/

In [7]:
# Downloading the pre-trained flan-t5 model
!transformers-cli download google/flan-t5-small

2023-05-07 23:48:32.788827: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at google/flan-t5-small were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# Load the required libraries
# from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset
import torch
import os
from tqdm import tqdm

In [9]:
# Load the pre-trained flan T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-small')
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')

In [10]:
# Load the Multi News dataset
dataset = load_dataset('multi_news')

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
# Extract the document and summary
documents = dataset['train']['document']
summary = dataset['train']['summary']

In [12]:
# Prepare the input and output for the flan T5 model
inputs = []
outputs = []
for i in range(len(documents)):
    input_text = f'Generate summary of: "{documents[i]}"'
    output_text = f'{summary[i]}'
    inputs.append(input_text)
    outputs.append(output_text)

In [ ]:
# Checking if GPU is available and setting device type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
# Tokenize the input and output using the T5 tokenizer
input_ids = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt')['input_ids'].to(device)
output_ids = tokenizer(outputs, padding=True, truncation=True, return_tensors='pt')['input_ids'].to(device)

In [14]:
# Fine-tune the flan T5 model on the Multi News dataset
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [15]:
# Actual Training
total_length_of_input = len(input_ids)
for i in tqdm(range(len(input_ids))):
    input_ids_batch = input_ids[i].unsqueeze(0).to(device)
    output_ids_batch = output_ids[i].unsqueeze(0).to(device)
    loss = model(input_ids_batch, labels=output_ids_batch).loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

100%|██████████| 44972/44972 [1:35:19<00:00,  7.86it/s]


In [ ]:
device = torch.device("cpu")
model.to(device)

In [25]:
# Generate summary based on the passage
passage = input("Enter the passage to summarize: ")
passage = passage.replace("\n", "")

# Set the input text and convert it to token ids
input_text = f'Generate summary of: "{passage}"'
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate the output with modified settings
output = model.generate(
    input_ids=input_ids,
    max_length=512,  # Increase max_length
    num_beams=8,  # Increase num_beams
    no_repeat_ngram_size=4,  # Increase no_repeat_ngram_size
    early_stopping=True,
    top_p=0.95, # Sample from the top 95% of the distribution
    temperature=0.5 # Control the level of randomness
)

# Convert the output token ids to text
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(output_text)

Enter the passage to summarize: According to popular myth, cockroaches would\ninherit a post nuclear disaster world. But it looks like the real winners might actually\nbe fungi. In 1999 fungi were found to be thriving in\nhighly radioactive conditions inside the Chernobyl reactor. These fungi are rich in melanin, yup, the\nsame stuff that gives you tan in summer. While radiation causes cell damage and mutations\nin most living creatures, melanin rich fungi are actually attracted to the stuff. And it seems the spherical nature of the melanin\nacts as a shield, but it also allows energy to be absorbed from the radiation, enhancing\nthe fungi’s growth. These fungi may one day help clean up radioactive\nwaste or even lead to the development of new cancer treatments. But if there is a nuclear accident in the\nmeantime, don’t panic right away. That mushroom cloud might not be what you\nthink it is. Hey, everyone, that is just a little segment\nI voiced for an Aussie TV show called Catalyst. 

In [26]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')

('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/spiece.model',
 'fine_tuned_model/added_tokens.json')

In [28]:
# Upload Model to Huggingface.co
# Install required dependencies
!pip install huggingface_hub -q

In [32]:
# Login with a token
from huggingface_hub import notebook_login, create_repo, HfApi
notebook_login()

In [40]:
# Create the repo
repo_name = "currentlyexhausted/flan-t5-summarizer"
create_repo(repo_name, repo_type="model")

RepoUrl('https://huggingface.co/currentlyexhausted/meow', endpoint='https://huggingface.co', repo_type='model', repo_id='currentlyexhausted/meow')

In [39]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_folder(
    folder_path="/content/fine_tuned_model",
    path_in_repo="./",
    repo_id=repo_name,
    repo_type="model"
)

'https://huggingface.co/currentlyexhausted/flan-t5-summarizer/tree/main/./'